In [222]:
# Import the libraries 

import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt

In [223]:
top_dests_ktm=pd.read_csv("../../Datasets/destinations_of_kathmandu_updated_with_latlong.csv")
print(top_dests_ktm.head())

                  title avg_rating  voted_by            genre    latitude  \
0      Boudhanath Stupa        4.5      8897  Religious Sites  27.7215062   
1  Swayambhunath Temple        4.5      6203  Religious Sites  27.7149298   
2  Pashupatinath Temple        4.5      4937  Religious Sites  27.7104461   
3     Chandragiri Hills        4.5       399        Mountains  27.6710496   
4       Kopan Monastery        4.5       787  Religious Sites  27.7425438   

   longitude  
0  85.359809  
1  85.288146  
2  85.346503  
3  85.262664  
4  85.362208  


In [224]:
# select the first 95 destinations as they're the destinations having the latitude and longitude currently
top_dests_ktm=top_dests_ktm[:95]
print(top_dests_ktm.tail())

                         title avg_rating  voted_by  \
90     Ghanta Ghar Clock Tower        3.5        11   
91  Jamchen Lhakhang Monastery        3.5         3   
92                    Big Bell        3.5         9   
93       Mrigasthali Deer Park          4         5   
94            Universal Crafts          5         3   

                              genre    latitude  longitude  
90   Points of Interest & Landmarks   27.707477  85.314711  
91                  Religious Sites  27.7215058  85.359192  
92   Points of Interest & Landmarks  27.7268583  85.300625  
93  Nature & Wildlife Areas • Parks   27.711512  85.349772  
94    Art Galleries • Antique Shops  27.7173331  85.381252  


In [225]:
# since we'll be dealing only with latitude and longitude at the moment
# only filter those columns along with the title
print(top_dests_ktm[['title','latitude','longitude']])

                         title    latitude  longitude
0             Boudhanath Stupa  27.7215062  85.359809
1         Swayambhunath Temple  27.7149298  85.288146
2         Pashupatinath Temple  27.7104461  85.346503
3            Chandragiri Hills  27.6710496  85.262664
4              Kopan Monastery  27.7425438  85.362208
..                         ...         ...        ...
90     Ghanta Ghar Clock Tower   27.707477  85.314711
91  Jamchen Lhakhang Monastery  27.7215058  85.359192
92                    Big Bell  27.7268583  85.300625
93       Mrigasthali Deer Park   27.711512  85.349772
94            Universal Crafts  27.7173331  85.381252

[95 rows x 3 columns]


In [226]:
# let's just pick n=10 for now for better visualization
n=90 #graph size
top_n_dests=top_dests_ktm[:n]
print(top_n_dests)

                     title avg_rating  voted_by  \
0         Boudhanath Stupa        4.5      8897   
1     Swayambhunath Temple        4.5      6203   
2     Pashupatinath Temple        4.5      4937   
3        Chandragiri Hills        4.5       399   
4          Kopan Monastery        4.5       787   
..                     ...        ...       ...   
85          Gaddhi Baithak          3        13   
86              Casino Rad          3        10   
87  Sundarijal Water Falls          5         7   
88     Shiva Mandir Temple        3.5        11   
89   Bagwati Mandir Temple        4.5         6   

                                                genre    latitude  longitude  
0                                     Religious Sites  27.7215062  85.359809  
1                                     Religious Sites  27.7149298  85.288146  
2                                     Religious Sites  27.7104461  85.346503  
3                                           Mountains  27.6710496  85.2

In [227]:
# also meanwhile let's create a matrix called 'graph' to store the weights for each edges
graph=np.zeros((n,n))

def create_graph():

    # iterators to iterate through the graph
    i=0
    j=0

    for lat,long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
        j=0
        for another_lat,another_long in zip(top_n_dests['latitude'].astype(float),top_n_dests['longitude'].astype(float)):
            #edge weight(Euclidean distance)
            distance_between_the_places=np.sqrt((lat-another_lat)**2+(long-another_long)**2)
            #print(distance_between_the_places)

            #and store it in the 'graph' matrix
            graph[i][j]=distance_between_the_places
            #increment j 
            j+=1

        #increment i
        i+=1

In [228]:
create_graph()
print(graph)

[[0.         0.07196422 0.01730239 ... 0.07007488 0.07756709 0.08449705]
 [0.07196422 0.         0.05852919 ... 0.1380494  0.00632343 0.01982038]
 [0.01730239 0.05852919 0.         ... 0.08726584 0.06360615 0.06893021]
 ...
 [0.07007488 0.1380494  0.08726584 ... 0.         0.14415738 0.15309451]
 [0.07756709 0.00632343 0.06360615 ... 0.14415738 0.         0.01453568]
 [0.08449705 0.01982038 0.06893021 ... 0.15309451 0.01453568 0.        ]]


In [229]:
from numpy import argmin

## Now let's define the functions for implementing the 2-opt 

#### In a nutshell, 2-opt improves the path obtained from the greedy approach

In [230]:
def _swap_2opt(route, i, k):
    """ 
        Swapping the route 
    """
    new_route = route[0:i]
    new_route.extend(reversed(route[i:k + 1]))
    new_route.extend(route[k + 1:])
    return new_route


In [231]:
def route_cost(graph, path):
    """
        Calculates the cost for the route. 
    """
    cost = 0
    for index in range(len(path) - 1):
        cost = cost + graph[path[index]][path[index + 1]]
    # add last edge to form a cycle.
    cost = cost + graph[path[-1], path[0]]
    return cost

In [232]:
# implement the 2-opt heuristic to improve the returned greedy route

def tsp_2_opt(graph, route):
    """
    Approximate the optimal path of travelling salesman according to 2-opt algorithm
    Args:
        graph: 2d numpy array as graph
        route: list of nodes

    Returns:
        optimal path according to 2-opt algorithm

    Examples:
        >>> import numpy as np
        >>> graph = np.array([[  0, 300, 250, 190, 230],
        >>>                   [300,   0, 230, 330, 150],
        >>>                   [250, 230,   0, 240, 120],
        >>>                   [190, 330, 240,   0, 220],
        >>>                   [230, 150, 120, 220,   0]])
        >>> tsp_2_opt(graph)
    """
    improved = True
    best_found_route = route
    best_found_route_cost = route_cost(graph, best_found_route)
    while improved:
        improved = False
        for i in range(1, len(best_found_route) - 1):
            for k in range(i + 1, len(best_found_route) - 1):
                new_route = _swap_2opt(best_found_route, i, k)
                new_route_cost = route_cost(graph, new_route)
                if new_route_cost < best_found_route_cost:
                    best_found_route_cost = new_route_cost
                    best_found_route = new_route
                    improved = True
                    break
            if improved:
                break
    return best_found_route_cost,best_found_route


### The following is the implementation of the greedy approach i.e. using the Nearest Neighbor Heuristic

In [233]:
data= list(range(0,n))# should contain the list of city ids 
matrix=graph

In [234]:
# redefining the nearest neighbour heuristic
import time
def nearest_neighbor_heuristic_route_generator(starting_point):
    """
        Greedy Nearest neighbor heuristic route generator
    """
    # maintain a list called 'visited' to store the cities that have been visited
    # initially, this holds the starting city
    visited=[]
    visited.append(starting_point)
    current_point=starting_point
    #also maintain a local copy of all the cities
    data_copy=data.copy()
    #remove the visited city from this list
    data_copy.remove(starting_point)
    #total cost of the route
    total_cost=0
    
    # from the starting city, calculate the cost to every cities(except itself) and select the city with minimum cost
    while data_copy !=[]:
        #calculate cost to every other cities in the cities_copy from the current city and store in cost_list
        cost_list=[]
        for neighbor_point in data_copy:
            cost=matrix[current_point][neighbor_point]
            cost_list.append(cost)
#         print(cost_list)
        min_idx=np.argmin(cost_list)#selects the index with the minimum cost
#         print(min_idx) 
        #also add the min cost to the total cost
        total_cost+=min(cost_list)
        #now find the corresponding point (neighbor)
        corresponding_point=data_copy[min_idx]
        # add that to the visited list
        visited.append(corresponding_point)
        # remove that from the data_copy list
        data_copy.remove(corresponding_point)
        # change current city to the city with min cost (corresponding city)
        current_point=corresponding_point
        
    # last city has to be connected to the starting city so let's do that
    total_cost+=matrix[current_point][starting_point] #add to total cost
    visited.append(starting_point)
    
    return total_cost,visited
        

#display the result for the data
t_in=time.time()
total_cost,route_generated=nearest_neighbor_heuristic_route_generator(0)
t_out1=time.time()


#apply 2-opt 
final_cost,final_route=tsp_2_opt(np.array(matrix),route_generated)
t_out2=time.time()
    

In [235]:
for i in route_generated:
    print(top_n_dests.iloc[i]["title"],end='')
    print("--->",end='')

Boudhanath Stupa--->Guru Lhakhang Monastery--->Boudha Stupa Thanka Center--->Boudha Farmers Market at Utpala Cafe--->Shechen Monastery--->Taragaon Museum--->Guhyeshwari Temple--->Pashupatinath Temple--->Aarya Ghat--->The Crematoria--->Aviation Museum--->Nepal Art Council Gallery--->Babar Mahal Revisted--->Kathmandu Fun Park--->Kashmiri Mosque--->Rani Pokhari (Queen's Pond)--->Ratna Park--->Asan--->Annapurna Temple--->Seto Machindranath Temple--->Akash Bhairav Temple--->Indra Chowk--->Makhan Tole--->Degu Taleju Temple--->Hanuman Dhoka--->The Tribhuvan, Mahendra, and Birendra Museums--->Kala Bairav--->Basantapur Tower--->Kathmandu Durbar Square--->Gaddhi Baithak--->Freak Street (Jhhonchen Tole)--->Basantpur Dabali--->Maru Ganesh Shrine--->Kashthamandap--->Bishal Bazaar--->Nara Devi Temple--->Thahiti Chowk--->himalayan arts gallery pltd--->Mandala Street--->Thamel--->Chhango Adventure Canyoning In Nepal--->Woodcraft Gallery--->Divine Yoga Studio--->Shanti Spa--->Astrek Climbing Wall--->Tr

In [236]:
#apply 2-opt 
final_cost,final_route=tsp_2_opt(np.array(matrix),route_generated)
print("Final cost:",final_cost)
print("Final route:",final_route)

Final cost: 1.6896352025677
Final route: [0, 50, 82, 23, 29, 20, 7, 11, 34, 32, 87, 66, 35, 22, 4, 43, 10, 77, 21, 13, 39, 54, 12, 36, 3, 89, 16, 26, 88, 38, 56, 52, 30, 1, 64, 78, 24, 41, 68, 57, 70, 61, 74, 9, 59, 42, 27, 25, 67, 58, 40, 14, 79, 46, 44, 85, 76, 60, 8, 19, 31, 63, 62, 72, 28, 5, 49, 75, 81, 65, 33, 71, 6, 51, 83, 86, 17, 55, 80, 69, 18, 84, 15, 45, 2, 53, 47, 37, 48, 73, 0]


In [237]:
for i in final_route:
    print(top_n_dests.iloc[i]["title"],end='')
    print("--->",end='')

Boudhanath Stupa--->Boudha Stupa Thanka Center--->Aviation Museum--->National Botanical Gardens--->Sankhu Village--->Phulchoki--->Namo Buddha (Stupa)--->Kailashnath Mahadev--->Templo de Changu Narayan--->Vajrayogini Temple--->Sundarijal Water Falls--->Gokarneshwor Mahadev Temple--->White Monastery--->Pullahari Monastery--->Kopan Monastery--->Nagi Gompa--->Shivapuri Nagarjun National Park--->Namkha Khyung Dzong Rigdin Choling Monastery--->Budhanilakantha Temple--->Budhanilkantha--->Nagarjun Forest Reserve--->Manakamana Mandir Temple--->Dakshinkali Temple--->Taudaha Lake--->Chandragiri Hills--->Bagwati Mandir Temple--->Kumari Chowk--->Kathesimbu Stupa--->Shiva Mandir Temple--->The National Museum--->Tribhuvan Park--->The Natural History Museum--->Amideva Buddha Park--->Swayambhunath Temple--->Military Museum--->Casino Mahjong--->Jaganath (Krishna) Temple--->Cathedral of the Assumption of the Blessed Virgin Mary--->Music Museum of Nepal--->Shiva Parvati Temple--->Bhimsen Tower--->Ratna Pa

In [238]:
#actual distance given by Vincenty distance using more accurate ellipsoidal models such as WGS-84 than Haversine
import geopy.distance
total_distance_without=0 #total actual distance

for i in range(len(route_generated)-1):
    coords_1 = (top_n_dests.iloc[route_generated[i]]["latitude"],top_n_dests.iloc[route_generated[i]]["longitude"])
    coords_2 = (top_n_dests.iloc[route_generated[i+1]]["latitude"],top_n_dests.iloc[route_generated[i+1]]["longitude"])
    
    #names of destinations connected
    src=top_n_dests.iloc[route_generated[i]]["title"]
    dest=top_n_dests.iloc[route_generated[i+1]]["title"]
    distance=geopy.distance.geodesic(coords_1, coords_2).km
    print (f'{str(src)+"->"+str(dest)}',distance)
    total_distance_without=total_distance_without+distance


Boudhanath Stupa->Guru Lhakhang Monastery 0.0579728509975531
Guru Lhakhang Monastery->Boudha Stupa Thanka Center 0.10895537810120313
Boudha Stupa Thanka Center->Boudha Farmers Market at Utpala Cafe 0.37358300701748093
Boudha Farmers Market at Utpala Cafe->Shechen Monastery 0.17521402809011333
Shechen Monastery->Taragaon Museum 0.6700413194108685
Taragaon Museum->Guhyeshwari Temple 1.0258861798307768
Guhyeshwari Temple->Pashupatinath Temple 0.4761235436934117
Pashupatinath Temple->Aarya Ghat 0.11154467192282523
Aarya Ghat->The Crematoria 0.3413347297505183
The Crematoria->Aviation Museum 1.4788977118405742
Aviation Museum->Nepal Art Council Gallery 2.947998502141368
Nepal Art Council Gallery->Babar Mahal Revisted 0.24376418914089013
Babar Mahal Revisted->Kathmandu Fun Park 0.8180516804712742
Kathmandu Fun Park->Kashmiri Mosque 0.856127884321023
Kashmiri Mosque->Rani Pokhari (Queen's Pond) 0.24671290409219834
Rani Pokhari (Queen's Pond)->Ratna Park 0.15658600750024554
Ratna Park->Asan 0.

In [239]:
#actual distance given by Vincenty distance using more accurate ellipsoidal models such as WGS-84 than Haversine
import geopy.distance
total_distance_with=0 #total actual distance

for i in range(len(final_route)-1):
    coords_1 = (top_n_dests.iloc[final_route[i]]["latitude"],top_n_dests.iloc[final_route[i]]["longitude"])
    coords_2 = (top_n_dests.iloc[final_route[i+1]]["latitude"],top_n_dests.iloc[final_route[i+1]]["longitude"])
    
    #names of destinations connected
    src=top_n_dests.iloc[final_route[i]]["title"]
    dest=top_n_dests.iloc[final_route[i+1]]["title"]
    distance=geopy.distance.geodesic(coords_1, coords_2).km
    print (f'{str(src)+"->"+str(dest)}',distance)
    total_distance_with=total_distance_with+distance


Boudhanath Stupa->Boudha Stupa Thanka Center 0.06371384026551553
Boudha Stupa Thanka Center->Aviation Museum 3.151947141557842
Aviation Museum->National Botanical Gardens 11.051795213745997
National Botanical Gardens->Sankhu Village 12.240871982297616
Sankhu Village->Phulchoki 10.969988763791966
Phulchoki->Namo Buddha (Stupa) 18.743477022959734
Namo Buddha (Stupa)->Kailashnath Mahadev 13.417091282781502
Kailashnath Mahadev->Templo de Changu Narayan 8.774620731552663
Templo de Changu Narayan->Vajrayogini Temple 4.913554333635855
Vajrayogini Temple->Sundarijal Water Falls 4.815274716869781
Sundarijal Water Falls->Gokarneshwor Mahadev Temple 3.9604695779604335
Gokarneshwor Mahadev Temple->White Monastery 2.4740081229374122
White Monastery->Pullahari Monastery 1.4334152272611844
Pullahari Monastery->Kopan Monastery 0.8250061123659818
Kopan Monastery->Nagi Gompa 4.817353450420873
Nagi Gompa->Shivapuri Nagarjun National Park 3.4262219665575535
Shivapuri Nagarjun National Park->Namkha Khyung 

In [240]:
print("without time:",t_out1-t_in)
print("with time:",t_out2-t_in)

print("Total distance (without 2-opt):",total_distance_without)
# print("Route generated (without 2-opt):",route_generated)
print("Final distance with 2-opt:",total_distance_with)
# print("Final route with 2-opt:",final_route)

without time: 0.00461888313293457
with time: 2.237117290496826
Total distance (without 2-opt): 198.86200669530433
Final distance with 2-opt: 177.66716838698292
